## load libraries and model


In [ ]:
import os.path as osp, os

os.environ["CUDA_VISIBLE_DEVICES"] = ""
import torch, polars as pl
from gnnepcsaft.data.graphdataset import ThermoMLDataset, Ramirez, ThermoMLpara
from gnnepcsaft.train.utils import create_model
from gnnepcsaft.train.utils import calc_deg
from gnnepcsaft.data.graph import from_InChI
from gnnepcsaft.configs.default import get_config
import pickle, numpy as np
from rdkit import Chem
from gnnepcsaft.demo.utils import plotdata, loadckp, model_para_fn, datacsv, plotparams

torch.cuda.is_available()

In [ ]:
ra_loader = Ramirez("gnnepcsaft/data/ramirez2022")
ra_para = {}
for graph in ra_loader:
    inchi, para = graph.InChI, graph.para.view(-1, 3).round(decimals=2)
    ra_para[inchi] = para.tolist()[0]
tml_loader = ThermoMLpara("gnnepcsaft/data/thermoml")
tml_para = {}
for graph in tml_loader:
    inchi, para = graph.InChI, graph.para.view(-1, 3).round(decimals=2)
    tml_para[inchi] = para.tolist()[0]
path = osp.join("gnnepcsaft/data", "thermoml")
testloader = ThermoMLDataset(path)
device = torch.device("cpu")

In [ ]:
config = get_config()
config.model = "PNA"
config.propagation_depth = 2
config.hidden_dim = 128
config.num_mlp_layers = 1
config.pre_layers = 1
config.post_layers = 3

In [ ]:
model_dtype = torch.float64
# Create and initialize the network.
deg_model2 = calc_deg("thermoml", "gnnepcsaft/")
model2 = create_model(config, deg_model2).to(device, model_dtype)
# Create and initialize the network.
deg_model1 = calc_deg("ramirez", "gnnepcsaft/")
model1 = create_model(config, deg_model1).to(device, model_dtype)
# Set up checkpointing of the model.

ckp_path = "gnnepcsaft/train/checkpoints/model6-8_25e6.pth"
loadckp(ckp_path, model2)
ckp_path = "gnnepcsaft/train/checkpoints/model5-8_25e6.pth"
loadckp(ckp_path, model1)

## evaluations


In [ ]:
model1_para, model1_array = model_para_fn(model1)
model2_para, model2_array = model_para_fn(model2)

In [ ]:
data2 = datacsv(model2_para)
data1 = datacsv(model1_para)
model1data = pl.DataFrame(data1)
model2data = pl.DataFrame(data2)
model1data.write_csv("model1.csv")
model2data.write_csv("model2.csv")

## plotting


In [ ]:
inchi = "InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3"
molecule_name = "UNKNOWN"
plotdata(inchi, molecule_name, [model1, model2])

In [ ]:
smiles = ["C" * i + "(=O)OCC" for i in range(1, 100)]

In [ ]:
smiles = ["C" * i for i in range(1, 100)]

In [ ]:
smiles = ["C" * i + "O" for i in range(1, 100)]

In [ ]:
plotparams(smiles, [model1, model2], "UNKNOWN")

In [ ]:
test_inchis = []
for gh in testloader:
    test_inchis.append(gh.InChI)
inchis = []
for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    inchi = Chem.MolToInchi(mol)
    if inchi in test_inchis:
        inchis.append(inchi)

In [ ]:
inchis